In [1]:
import json
import yaml
import zipfile

In [2]:
with open('data/data.json') as f:
    data = json.load(f)

In [3]:
dic = data[0]

In [4]:
dic

{'_id': {'$oid': '666bde9ece6dd07810e474a8'},
 'datetime': {'$date': '2024-06-14T08:09:18.224Z'},
 'campaign_id': 'C007',
 'campaign_description': 'Testing Boundary Finder',
 'operator_name': 'Jon',
 'series_id': 'C007_S002',
 'series_description': 'testing save ok',
 'recipe_name': 'none',
 'sdl_algorithm': 'EE_LearnMinimumRate',
 'samples_produced': False,
 'qcms_active': True,
 'source_1_material': 'Zr',
 'source_1_supply': 1,
 'source_1_mode': 'pulsed DC',
 'source_1_ramp_rate_[W/s]': 10,
 'source_2_material': 'CuS',
 'source_2_supply': 2,
 'source_2_mode': 'pulsed DC',
 'source_2_ramp_rate_[W/s]': 5,
 'source_3_material': 'Sn',
 'source_3_supply': 3,
 'source_3_mode': 'pulsed DC',
 'source_3_ramp_rate_[W/s]': 10,
 'source_4_material': 'SnS',
 'source_4_supply': 4,
 'source_4_mode': 'pulsed DC',
 'source_4_ramp_rate_[W/s]': 5,
 'source_5_material': 'Ba',
 'source_5_supply': 5,
 'source_5_mode': 'RF',
 'source_5_ramp_rate_[W/s]': 5,
 'source_6_material': 'ZnS',
 'source_6_supply': 6

In [5]:
def getPoweredAxes(arr):
    axes = []
    for i in range(len(arr)):
        if arr[i]:
            axes.append(str(i+1))
    return axes

In [6]:
getPoweredAxes([True,True,True])

['1', '2', '3']

In [ ]:
def getYamlMagnetrons(dic, power_axes):
    axes = getPoweredAxes(power_axes)
    return [getYamlMagnetron(dic, axis) for axis in axes]

In [ ]:
def getYamlMagnetron(dic, axis):
    magnetron = {
        'name' : dic['source_' + axis + '_material'],
        'material' : {
            'name' : dic['source_' + axis + '_material'],
            'lab_id' : None
        },
        'vapor_source' : {
            'setpoints' : {
            'set_power' : dic['source_' + axis + '_set_power_[W]'],
            'set_voltage' : None,
            'set_current' : None
            },
            'power' : {
                'values' : dic['Actual_Power_Ax' + axis + '_[W]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Actual_Power_Ax' + axis + '_[W]_Mean'],
                'error' : dic['Actual_Power_Ax' + axis + '_[W]_STD']
            },
            'voltage' : {
                'values' : dic['Voltage_Ax' + axis + '_[V]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Voltage_Ax' + axis + '_[V]_Mean'],
                'error' : dic['Voltage_Ax' + axis + '_[V]_STD'],
            },
            'current' : {
                'values' : [],
                'times' : dic['Time_[s]'],
                'mean' : None,
                'error' : None
            },
            'power_supply' : {
                'instrument_id' : "Power Supply Ax" + axis,
                'frequency' : None
            }
        },
        'position' : {
            'center_xyz' : None,
            'center_normal' : None,
            'rotation' : None
        }
    }
    return magnetron

In [ ]:
arr = [True, True, True]

        
#getYamlMagnetrons(dic, arr)
        

# Environment

In [ ]:
def getYamlQCM(dictionary, i):
    prefix = 'QCM_' + str(i) + '_'
    yamlQCM = {
            'crystal_info' : {
                'info' : None,
                'resonant_frequency' : None
            },
            'sensor_data' : {
                'values' : dictionary[prefix + 'frequency_[Hz]'],
                'mean' : None,
                'error' : None,
                'slope' : dictionary[prefix + 'rate_[df/dt]']
            },
            'remaining_lifetime' : {
                'value' : dictionary[prefix + 'xtal_lifetime_[%]']
            },
            'mass_deposition_rate' : {
                'value' : None,
                'error' : None
            }
        }
    return yamlQCM

In [ ]:
def getYamlEnv(dictionary):
    environment = {
        'gas_flow' : {
            'values' :  dictionary['Gas_flow_[sccm]'],
            'mean' : dictionary['Gas_flow_[sccm]_Mean'],
            'error' : dictionary['Gas_flow_[sccm]_STD'],
            'gas' : 'Here we do the pubchem thing'
        },
        'setpoints' : {
            'pressure' : dictionary['Pressure_setpoint_[mTorr]'],
            'flow' : None,
        },
        'pressure' : {
            'values' : dictionary['Actual_pressure_[mTorr]'],
            'mean' : dictionary['Actual_pressure_[mTorr]_Mean'],
            'error' : dictionary['Actual_pressure_[mTorr]_STD']
        },
        'sensors' : [getYamlQCM(dictionary, i+1) for i in range(3)]
    }
    return environment
    


In [ ]:
#environment = getYamlEnv(dic)

In [ ]:
data = {'data' : {
            'm_def' : "../uploads/1WE7CfSoQbS8LidbT0XLfw/raw/automate-solar_schema.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition",
            'quantity' : None,
            'name' : dic['Run_ID'],
            'lab_id' : "Uppsala University Åutomate-Solar",
            'description' : f'{dic["Campaign description"]}, {dic["Campaign code"]}, {dic["Series_description"]}',
            'steps' : [
                {
                    'sources' : getYamlMagnetrons(dic, [True,True,True]),
                    'environment' : getYamlEnv(dic)
                }
            ]
    }
}

In [ ]:
#print(yaml.dump(data, default_flow_style=False))

In [ ]:
with open('data/data.archive.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
with zipfile.ZipFile('data/data.zip', 'w') as zipped_f:
    zipped_f.writestr("data/data.archive.yaml", yaml.dump(data, default_flow_style=False))

In [ ]:
#test = yaml.dump(data, default_flow_style=False)